In [ ]:
import sklearn
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import math
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))

dataframe=pd.read_csv('Data/dengue_features_train.csv')
labels=pd.read_csv('Data/dengue_labels_train.csv')
target_df=pd.read_csv('Data/dengue_features_test.csv')
submission_format=pd.read_csv('Data/submission_format.csv')
#target_df['month']=[int(i[5:7]) for i in target_df['week_start_date']]

def clean_data(df):
    #file.set_index('week_start_date', drop = True, inplace = True)
    #df['month']=[int(i[5:7]) for i in df['week_start_date']]
    #df['nvdi']= df['ndvi_ne']+df['ndvi_nw']+df['ndvi_se']+df['ndvi_sw']
    
    df=df.drop(['city','week_start_date', 'year' ], axis = 1, inplace = True)
    
    #,'ndvi_ne','ndvi_nw', 'ndvi_se', 'ndvi_sw', 'reanalysis_tdtr_k', 'precipitation_amt_mm', 'reanalysis_sat_precip_amt_mm','reanalysis_max_air_temp_k'

def fill_missing_values(df):
    imputer = SimpleImputer(strategy="median") 
    imputer.fit(df)
    x=imputer.transform(df)
   
    return pd.DataFrame(x, columns=df.columns)

def split_data_in_cities(df):
    #cities=set(df['city'])
    cities=['sj','iq']
    new_files=[]
    for city in list(cities):
        new_files.append(df[df['city'] == city])

    return new_files

def eliminate_not_outliers(df_labels, city_df):
    cases=np.array(df_labels['total_cases'])
    df_labels=df_labels.reset_index()
    city_df=city_df.reset_index()
    rows_to_eliminate=[]
    mean=cases.mean()
    for i in range(len(cases)):
        if(cases[i]<(mean+20)):
            rows_to_eliminate.append(i)
    
    #print(rows_to_eliminate)
    #print(df_labels)
    df_labels.drop(rows_to_eliminate, axis=0, inplace=True)
    city_df.drop(rows_to_eliminate, axis=0, inplace=True)
    df_labels.drop(['index'], axis=1, inplace=True)
    city_df.drop(['index'], axis=1, inplace=True)
    return df_labels, city_df

def preprocess_data(df):
    clean_data(df)
    df=fill_missing_values(df)
    return df

def save_submission(first_prediction, second_prediction, name_of_file):
    solution=list(first_prediction)
    for x in second_prediction:
        solution.append(x)
        
    solution=np.array(solution)
    solution=np.int64(np.round(solution))
    submission_format['total_cases']=pd.DataFrame(solution)[0]
    submission_format.to_csv(name_of_file, index=False)
    
    
new_files=split_data_in_cities(dataframe)
new_labels=split_data_in_cities(labels)
new_target=split_data_in_cities(target_df)

sj_df=new_files[0]
iq_df=new_files[1]

sj_labels=new_labels[0]
iq_labels=new_labels[1]

sj_target=new_target[0]
iq_target=new_target[1]

sj_df=preprocess_data(sj_df)
iq_df=preprocess_data(iq_df)
sj_target=preprocess_data(sj_target)
iq_target=preprocess_data(iq_target)
#clean_data(sj_df)
#clean_data(iq_df)
#sj_labels, sj_df =eliminate_not_outliers(sj_labels, sj_df)
#iq_labels, iq_df= eliminate_not_outliers(iq_labels, iq_df)

#sj_df=fill_missing_values(sj_df)
#iq_df=fill_missing_values(iq_df)

#clean_data(dataframe)
#dataframe=fill_missing_values(dataframe)

sj_df

In [ ]:


regression_models = [RandomForestRegressor(),
                     ExtraTreesRegressor()]

names = ["RandomForestRegressor",
         "ExtraTreesRegressor"]

complexity_evaluator = ComplexityEvaluator.ComplexityEvaluator(
    [500, 1000, 2000, 5000],
    [5, 10, 20, 50, 100, 200])

i = 0
for model in regression_models:
    res, data = complexity_evaluator.Run(model, random_data_regression)
    print(names[i])
    print tabulate(data, headers='keys', tablefmt='psql', showindex='never')
    i = i + 1
